# Build and Register an MXNet Image Classification Model via SageMaker Pipelines

In [2]:
!pip install --upgrade pip
!pip install --upgrade sagemaker

  Using cached sagemaker-2.168.0-py2.py3-none-any.whl
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.6.0
    Uninstalling importlib-metadata-6.6.0:
      Successfully uninstalled importlib-metadata-6.6.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.165.0
    Uninstalling sagemaker-2.165.0:
      Successfully uninstalled sagemaker-2.165.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-sub

## 1. Create SageMaker session and client

In [3]:
import os
import logging
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
prefix = 'vy-retry-with-lambda'

sm_client = boto3.Session().client(service_name="sagemaker", region_name=region)

In [5]:
# Upload data - only need to do once!

s3_image_folder = "s3://{}/{}/image-structure/".format(bucket, prefix)
print(s3_image_folder)

s3://sagemaker-us-east-1-233328792017/vy-retry-with-lambda/image-structure/

The user-provided path image_structure does not exist.


## 2. Define SageMaker Pipeline parameters

In [4]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)


# Setting a SageMaker Pipeline Session is important to avoid pipeline steps from running before the pipeline is ready
sm_pipeline_session = PipelineSession(boto_session=boto_session, sagemaker_client=sm_client, default_bucket=bucket)

model_package_group_name = "retry-lambda-Image-Classification"  # Model name in model registry
pipeline_name = "RetryLambdaImageClassificationPipeline" 

###### TODO ######
input_img_data_s3_uri = "s3://{}/{}".format(bucket, prefix)

input_data = ParameterString(
    name="InputData",
    default_value=input_img_data_s3_uri
)
print(input_img_data_s3_uri)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value = "ml.t3.medium")
training_instance_type = ParameterString(name="TrainingInstanceType", default_value = "ml.p2.xlarge")

train_split_percentage = ParameterFloat(
    name="TrainSplitPercentage",
    default_value=0.75
)

validation_split_percentage = ParameterFloat(
    name="ValidationSplitPercentage",
    default_value=0.10
)

test_split_percentage = ParameterFloat(
    name="TestSplitPercentage",
    default_value=0.15
)


s3://sagemaker-us-east-1-547381887603/vy-retry-with-lambda


## 3. Define image preprocessing step

In this ML workflow step, you will be converting the raw .jpg image files in the S3 input bucket to the Apache MXNet RecordIO format, which is the recommended input format for the Amazon SageMaker image classification algorithm. Upon completion, the newly generated .rec files will be split into train, validation, and test sets and uploaded back to the original S3 input bucket under a newly created `recordIO` folder. This step relies on the `preprocess.py` script found in the `scripts` directory.

In [5]:
%%writefile retry_preprocess.py

#!/usr/bin/python3.7

import argparse
import os
import logging
import subprocess


logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


if __name__ == "__main__":
    
    ###### 1. Parse input arguments ######
    parser = argparse.ArgumentParser()
    parser.add_argument('--input-s3-bucket', type=str, dest='INPUT_S3_BUCKET')
    parser.add_argument('--train-split-percentage', type=str, default=0.70, dest='TRAIN_SPLIT_PERCENTAGE')
    parser.add_argument('--validation-split-percentage', type=str, default=0.15, dest='VALIDATION_SPLIT_PERCENTAGE')
    parser.add_argument('--test-split-percentage', type=str, default=0.15, dest='TEST_SPLIT_PERCENTAGE')

    args = parser.parse_args()
    print(args.INPUT_S3_BUCKET)
    
    ###### 2. Create RecordIO files from image files ######
    logger.debug("Creating RecordIO files from image files")
    
    DATASET_NAME="image-structure"
    BASE_DIR="/opt/ml/processing/input/data"
    IM2REC_PATH="/opt/ml/processing/input/code/im2rec.py"
    
    command = f"cd {BASE_DIR};\
            pwd;\
            ls;\
            rm *.rec;\
            echo 'Downloading im2rec.py script from Apache MXNet';\
            wget -P /opt/ml/processing/input/code/ https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py;\
            cd /opt/ml/processing/input/code/;\
            pwd;\
            cd {BASE_DIR};\
            echo 'Creating LST files';\
            python {IM2REC_PATH} --list --recursive --pass-through --test-ratio={args.TEST_SPLIT_PERCENTAGE} --train-ratio={args.TRAIN_SPLIT_PERCENTAGE} {DATASET_NAME} {DATASET_NAME} > {DATASET_NAME}_classes;\
            echo 'Label classes:';\
            cat {DATASET_NAME}_classes;\
            python {IM2REC_PATH} --num-thread=4 {DATASET_NAME}_train.lst {DATASET_NAME};\
            python {IM2REC_PATH} --num-thread=4 {DATASET_NAME}_val.lst {DATASET_NAME};\
            python {IM2REC_PATH} --num-thread=4 {DATASET_NAME}_test.lst {DATASET_NAME};\
            ls -lh *.rec"
    
    ret = subprocess.run(command, capture_output=True, shell=True)
    print(ret.stdout.decode())

    ###### 3. Upload RecordIO files to S3 ######
    logger.debug("Uploading newly created RecordIO back to S3")

    # Upload our train and test RecordIO files back to S3 bucket    
    TRAIN_CHANNEL = '/opt/ml/processing/train'
    VALIDATION_CHANNEL = '/opt/ml/processing/validation'
    TEST_CHANNEL = '/opt/ml/processing/test'

    # Clean up any existing data in S3
    os.system(f"aws s3 rm {args.INPUT_S3_BUCKET}/recordIO/train --recursive")
    os.system(f"aws s3 rm {args.INPUT_S3_BUCKET}/recordIO/validation --recursive")
    os.system(f"aws s3 rm {args.INPUT_S3_BUCKET}/recordIO/test --recursive")
    
    # Upload the rec files to the train, validation, test channels
    os.system(f"cp /opt/ml/processing/input/data/{DATASET_NAME}_train.rec {TRAIN_CHANNEL}")
    os.system(f"cp /opt/ml/processing/input/data/{DATASET_NAME}_val.rec {VALIDATION_CHANNEL}")
    os.system(f"cp /opt/ml/processing/input/data/{DATASET_NAME}_test.rec {TEST_CHANNEL}")
    
    print("Done!")


Overwriting retry_preprocess.py


In [6]:
from sagemaker.mxnet import MXNetProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


mxnet_processor_preprocess = MXNetProcessor(
    framework_version="1.8.0",
    py_version="py37",
    instance_type=processing_instance_type.default_value,
    instance_count=processing_instance_count.default_value,
    base_job_name=f"{prefix}/preprocess-image-data",
    sagemaker_session=sm_pipeline_session,
    role=role
)

processing_inputs = [
    ProcessingInput(
        input_name="input_img_data", 
        source=input_data,
        destination="/opt/ml/processing/input/data"
    )
]

processing_outputs = [
    ProcessingOutput(
        output_name="train", 
        source="/opt/ml/processing/train",
        destination=f"{input_data.default_value}/recordIO/train"
    ),
    ProcessingOutput(
        output_name="validation", 
        source="/opt/ml/processing/validation",
        destination=f"{input_data.default_value}/recordIO/validation"
    ),
    ProcessingOutput(
        output_name="test", 
        source="/opt/ml/processing/test",
        destination=f"{input_data.default_value}/recordIO/test"
    )
]

step_args = mxnet_processor_preprocess.run(
    code="retry_preprocess.py",
    inputs=processing_inputs,
    outputs=processing_outputs,
    arguments=[
        "--input-s3-bucket", 
        input_data.default_value,
        "--train-split-percentage",
        str(train_split_percentage.default_value),
        "--validation-split-percentage",
        str(validation_split_percentage.default_value),
        "--test-split-percentage",
        str(test_split_percentage.default_value)
    ]
)

step_preprocess = ProcessingStep(
    name="Preprocess-Image-Data",
    step_args=step_args,
)


/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:289: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


## 4. Define model training step

In this ML workflow step, you will train an MXNet image classification model using the train and validation .rec files that were created in the previous step. For more information regarding the specific image classification algorithm, refer to [Image Classification - MXNet](https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html) from the Amazon SageMaker documentation.

In [7]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


model_output_path = f"s3://{bucket}/{prefix}/model-output"

image_uri = sagemaker.image_uris.retrieve(
    region=region,
    framework="image-classification"
)

mxnet_train = Estimator(
    image_uri=image_uri,
    instance_type="ml.p2.xlarge",
    instance_count=1,
    volume_size=50,
    max_run=36000,
    output_path=model_output_path,
    sagemaker_session=sm_pipeline_session,
    role=role
)

# Feel free to edit these model hyperparameters based on domain expertise
# For more information regarding image classification hyperparameters, refer to https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html
mxnet_train.set_hyperparameters(
    use_pretrained_model=1,
    image_shape='3,100,100',
    num_classes=18,
    num_training_samples=31500, 
    learning_rate=0.1,
    mini_batch_size=25,
    epochs=1,
    early_stopping=True
)

train_data = sagemaker.inputs.TrainingInput(
    s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.inputs.TrainingInput(
    s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

step_args = mxnet_train.fit(
    inputs={
        "train": train_data, 
        "validation": validation_data
    },
    logs=True
)

step_train = TrainingStep(
    name="Train-Image-Classification-Model",
    step_args=step_args
)


## 5. Define model evaluation step

In this ML workflow step, you will be evaluating the trained model (from the previous step) on the test .rec file. Specifically, you will be measuring the accuracy and F1 score on the test set. This step relies on the `evaluate.py` script found in the `scripts` directory.

In [8]:
%%writefile retry_evaluation.py
#!/usr/bin/python3.7

import json
import logging
import pathlib
import pickle
import tarfile
import glob
import mxnet as mx

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


if __name__ == "__main__":
    logger.debug("Starting evaluation.")
    
    ###### 1. Loading trained MXNet model ######
    logger.debug("Loading mxnet model.")
    
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    
    param_file = glob.glob('./*.params')
    epoch = int(param_file[0][-11:-7])
    sym, arg_params, aux_params = mx.model.load_checkpoint("image-classification", epoch)
    
    model_shapes_json_file = open('./model-shapes.json')
    model_shapes_dict = json.load(model_shapes_json_file)[0]
    train_batch_size = model_shapes_dict['shape'][0]
    train_data_shape = tuple(model_shapes_dict['shape'][1:])

    ###### 2. Loading and preparing test .rec file ######
    logger.debug("Reading test data.")
    
    DATASET_NAME="image-structure"
    test_path = f"/opt/ml/processing/test/{DATASET_NAME}_test.rec"
    
    test = mx.io.ImageRecordIter(path_imgrec=test_path,
                                 data_name='data',
                                 label_name='softmax_label',
                                 batch_size=train_batch_size,
                                 data_shape=train_data_shape,
                                 rand_crop=False,
                                 rand_mirro=False)
    
    
    ###### 3. Making predictions on the test set ######
    logger.info("Performing predictions against test data.")
    
    mod = mx.mod.Module(symbol=sym, context=mx.cpu())
    
    mod.bind(for_training=False,
             data_shapes=test.provide_data,
             label_shapes=test.provide_label)
    
    mod.set_params(arg_params, aux_params)

    logger.debug("Calculating accuracy on test set.")
    metric = mod.score(eval_data=test, eval_metric=['acc'])
    test_accuracy = metric[0][1]
    
    print(f"Test Accuracy: {test_accuracy}")

    
    report_dict = {
        "classification_metrics": {
            "accuracy": {
                "value": test_accuracy
            }
        },
    }

    ###### 5. Saving evaluation metrics to output path ######
    logger.info("Writing out evaluation report")
    
    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)


    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))
        
    print("Done!")


Overwriting retry_evaluation.py


In [9]:
from sagemaker.workflow.properties import PropertyFile

mxnet_processor_eval = MXNetProcessor(
    framework_version="1.8.0",
    py_version="py37",
    instance_type=processing_instance_type.default_value,
    instance_count=processing_instance_count.default_value,
    base_job_name=f"{prefix}/model-evaluation",
    sagemaker_session=sm_pipeline_session,
    role=role
    
)

processing_inputs = [
    ProcessingInput(
        source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        destination="/opt/ml/processing/model"
    ),
    ProcessingInput(
        source=step_preprocess.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
        destination="/opt/ml/processing/test")
]

processing_outputs = [
    ProcessingOutput(
        output_name="evaluation", 
        source="/opt/ml/processing/evaluation",
        destination=f"s3://{bucket}/{prefix}/model-evaluation")
]

step_args=mxnet_processor_eval.run(
    code="retry_evaluation.py",
    inputs=processing_inputs,
    outputs=processing_outputs
)

evaluation_report = PropertyFile(
    name="Image-Classification-Model-Evaluation-Report",
    output_name="evaluation",
    path="evaluation.json"
)

step_eval = ProcessingStep(
    name="Evaluate-Image-Classification-Model",
    step_args=step_args,
    property_files=[evaluation_report]
)


## 6. Define model registering step

In this ML workflow step, you will conditionally register the trained model into SageMaker model registry only if the trained model accuracy (on the test set) is greater than or equal to 70%.



In [17]:
%%writefile lambda_helper.py

import json
import boto3

def lambda_handler(event, context):

    sm_client = boto3.client("sagemaker")

    # The name of the model created in the Pipeline CreateModelStep
    model_name_a = event["model_name_a"]
    model_name_b = event["model_name_b"]

    endpoint_config_name = event["endpoint_config_name"]
    endpoint_name = event["endpoint_name"]

    create_endpoint_config_response = sm_client.create_endpoint_config(
 
        EndpointConfigName = endpoint_config_name,
        ProductionVariants = [
            
            {
                "InstanceType": "ml.m4.xlarge",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "ModelName": model_name_a,
                "VariantName": "variant-a-50"
            },
            
            {
                "InstanceType": "ml.m4.xlarge",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "ModelName": model_name_b,
                "VariantName": "variant-b-50"
            }
            
        ]
        
    )

    create_endpoint_response = sm_client.create_endpoint(
        
        EndpointName = endpoint_name,
        EndpointConfigName = endpoint_config_name
        
    )

    return {
        "statusCode": 200,
        "body": json.dumps("Created Endpoint!"),
        "other_key": "example_value"
    }

Overwriting lambda_helper.py


In [18]:
# Register step:
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
import time
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda


model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)

model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sm_pipeline_session,
    role=role,
)

# #Create model
step_create_model = ModelStep(
    name="CreateModel",
    step_args=model.create("ml.m5.xlarge"),
)


#Register model:
step_args = model.register(
    content_types=["image/png"],
    response_types=["application/json"],
    inference_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics
)

step_register = ModelStep(
    name = "Register-Image-Classification-Model",
    step_args = step_args
)


INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/vy-retry-with-lambda/model-evaluation-2023-06-25-04-31-59-571/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/vy-retry-with-lambda/model-evaluation-2023-06-25-04-31-59-571/source/runproc.sh


Using provided s3_resource


In [19]:
# Lambda step:

current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
model_name = "project-lambda-image-model-" + current_time
endpoint_config_name = "project-lambda-deploy-image-endpoint-config-" + current_time
endpoint_name = "project-lambda-deploy-image-endpoint-" + current_time

function_name = "sagemaker-lambda-step-endpoint-deploy-image-model-" + current_time

func = Lambda(
    
    function_name = function_name,
    execution_role_arn = role,
    script = "lambda_helper.py",
    handler = "lambda_helper.lambda_handler"

)

output_param_1 = LambdaOutput(output_name = "statusCode", output_type = LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name = "body", output_type = LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name = "other_key", output_type = LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    
    name = "LambdaStep",
    lambda_func = func,
    inputs = {
        
        "model_name_a": step_create_model.properties.ModelName,
        "model_name_b": step_create_model.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name
        
    },
    outputs = [output_param_1, output_param_2, output_param_3]
    
)


In [20]:
# Create a condition to register the model if the model accuracy is greater than 0.70
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="classification_metrics.accuracy.value"
    ),
    right=0.10,
)

# This step encompasses 'step_register' and only performs the 'step_register' if the model accuracy is greater than 0.70
step_cond = ConditionStep(
    name="Check-Accuracy-Image-Classification-Model",
    conditions=[cond_gte],
    if_steps=[step_create_model, step_register, step_deploy_lambda],
    else_steps=[],
)


## 7. Create SageMaker Pipeline

In this step, you define a SageMaker pipeline encompassing all the above ML workflow steps.

In [21]:
from sagemaker.workflow.pipeline import Pipeline
import json


pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        input_data,
        processing_instance_count,
        processing_instance_type,
        train_split_percentage,
        validation_split_percentage,
        test_split_percentage
    ],
    steps=[
        step_preprocess, 
        step_train, 
        step_eval, 
        step_cond
    ],
    sagemaker_session=sm_pipeline_session
)


In [15]:
import json
 
json.loads(pipeline.definition())

INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/e9e9eb774b395df58e988c12afbe1592/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/19cb7f4656d787bdcecd7529d79c9bd2/runproc.sh


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/b5aa3c0ef7403c958f125bf003dce9f1/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/895a00bb866033f5a6af93e63f81c11c/runproc.sh


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-547381887603/vy-retry-with-lambda'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.t3.medium'},
  {'Name': 'TrainSplitPercentage', 'Type': 'Float', 'DefaultValue': 0.75},
  {'Name': 'ValidationSplitPercentage', 'Type': 'Float', 'DefaultValue': 0.1},
  {'Name': 'TestSplitPercentage', 'Type': 'Float', 'DefaultValue': 0.15}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Preprocess-Image-Data',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.t3.medium',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '763104351884.dkr.ecr

## 8. Start SageMaker Pipeline execution

In [22]:
# Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does
pipeline.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/e9e9eb774b395df58e988c12afbe1592/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/19cb7f4656d787bdcecd7529d79c9bd2/runproc.sh


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/b5aa3c0ef7403c958f125bf003dce9f1/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/895a00bb866033f5a6af93e63f81c11c/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/e9e9eb774b395df58e988c12afbe1592/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/19cb7f4656d787bdcecd7529d79c9bd2/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/b5aa3c0ef7403c958f125bf003dce9f1/sourcedir.tar.gz


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-547381887603/RetryLambdaImageClassificationPipeline/code/895a00bb866033f5a6af93e63f81c11c/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:547381887603:pipeline/RetryLambdaImageClassificationPipeline',
 'ResponseMetadata': {'RequestId': '19a73e2f-0b6e-4a68-a16f-d231aff64641',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '19a73e2f-0b6e-4a68-a16f-d231aff64641',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '106',
   'date': 'Sun, 25 Jun 2023 04:32:23 GMT'},
  'RetryAttempts': 0}}

In [23]:
# Start a pipeline execution
execution = pipeline.start()